## promise patterns


### resolve a promise property?

 TODO: make this even simpler, if working with promises then it can callback on it own similar to lazy loading



In [ ]:

function promiseOrResolve(obj, property, cb) {
    return obj ? (typeof obj[property] != 'undefined'
                  ? Promise.resolve(obj[property])
                  : cb(obj).then(d => (obj[property] = d)))
        : Promise.resolve()
}

module.exports = promiseOrResolve;


## classes and objects

Long term goal: use this notebook at training data to mangle/uglify/transpile any language

TODO: import falsey javascript in to this notebook?

demonstrate patterns between different languages, in C# interfaces are like:

```
interface MyInterface {
    myProperty: string
}

class MyClass : MyInterface {
}
```




### making a class the normal way?


In [ ]:
if(typeof parentClass === 'undefined') {
    class parentClass {
        constructor() {
            console.log('parent');
        }

        do_message() {
            console.log('original');
        }
    }

    class childClass extends parentClass {
        constructor() {
            console.log('child');
            super();
        }

        do_message() {
            super.do_message();
            console.log('overridden');
        }
    }
}

// TODO: better way to do this, childClass.prototype.bind?
module.exports = () => new childClass;

if(typeof $$ !== 'undefined') {
    console.log(typeof module.exports);
    var testClass = module.exports();
    testClass.do_message();
    
    /* expected output
    function
    child
    parent
    original
    overridden
    */
    
    /*
    this is extremely annoying, how to achieve the same effect with prototype?
    the javascript language is apparently not indempotent
    
    evalmachine.<anonymous>:1
    class parentClass {
    ^

    SyntaxError: Identifier 'parentClass' has already been declared
        at evalmachine.<anonymous>:1:1
    */
}




### making a class with prototype

derived from to help avoid annoying notebook errors:

https://stackoverflow.com/questions/10430279/extending-an-object-in-javascript


In [ ]:
var parentObj = Object.create({
    do_init() {
        console.log('parent');
    },
    do_message() {
        console.log('original');
    }
})

var childObj = Object.create(parentObj);
Object.assign(childObj, {
    do_init() {
        console.log('child');
        parentObj.do_init();
    },
    do_message() {
        parentObj.do_message();
        console.log('overridden');
    }
})

function extend(child, parent) {
    var newClass = Object.create(parent);
    Object.assign(newClass, child);
    var self = Object.create(newClass);
    return self;
}

function override(child) {
    return extend(child, this);
}

module.exports = {
    extend,
    override
}
//module.exports = Object.create.bind(null, childObj)

if(typeof $$ !== 'undefined') {
    console.log(typeof module.exports)
    var testClass = module.exports();
    testClass.do_init();
    testClass.do_message();
    
    /* expected output
    child
    parent
    original
    overridden
    */
}



### enforcing an interface?


In [ ]:
// in pre-prototype days objects were treated as classes

function applyInterface(class) {
    var properties = ['myProperty'];
    var i, result = {};
    for(i=0; i<properties.length; i++) {
        if(typeof class[properties[i]] === 'undefined') {
            throw new Error(`Missing interface property ${properties[i]}`);
        }
        result[properties[i]] = class[properties[i]];
    }
}

// newer ES6 syntax

function typeErrorTemplate(k, t, i) {
    var p = i.constructor;
    return `type mis-match ${k} "${typeof t}" is not "${typeof i}" of type ${p}`
}

function resolveType(compare, interface) {
    var type = compare === void 0 || compare === null
        ? compare
        : Object.getPrototypeOf(compare);
    var expected = interface === void 0 || interface === null
        ? interface
        : Object.getPrototypeOf(interface);
    switch(expected) {
        case Array.prototype:
            // if it is an empty array to the actual compare on the object,
            //   if it is not an empty array compare to each type in the array
            if(interface.length > 0) {
                return interface.filter(i => resolveType(compare, i)).length > 0;
                break;
            }
        default:
            if(type === expected) return true;
            if(!type || !expected || !(expected.isPrototypeOf(type))) {
                return false
            }
            return true;
    }
}

function interface(obj, interface) {
    var result = {};
    for(var i in interface) {
        if(interface.hasOwnProperty(i)) {
            // TODO: expand on this for inheritable objects and multiple values
            if(resolveType(obj[i], interface[i])) {
                console.log('matched ' + i)
                result[i] = obj[i];
            } else {
                throw new Error(typeErrorTemplate(i, obj[i], interface[i]))
            }
        }
    }
    return result;
}

module.exports = interface;

if(typeof $$ !== 'undefined') {
    var jsonInterface = {
        display_name: '',
        argv: [], // not optional
        language: '', // not optional
        metadata: [void 0, {}], // optional
        env: [void 0, {}], // TODO dictionary descriptor types?
        interrupt_mode: [void 0, '']
    }

    console.log(JSON.stringify(interface({
        display_name: 'Node JS',
        argv: [],
        language: '',
        some_other_stuff: true
    }, jsonInterface)))
    
    // expected output {"display_name":"Node JS","argv":[],"language":""}
}



#### test enforcing an interface with prototype?



In [63]:

function myInterface(overrides) {
    var interface = Object.create({});
    // TODO: wish there was a way to not name this twice
    Object.assign(interface, {
        propertyOne: overrides.propertyOne,
        print: overrides.print
    });
    return interface;
}

function print() {
    console.log(this.propertyOne);
    console.log(this.propertyTwo);
}

var myClass = Object.create({
    propertyOne: 'original 1',
    propertyTwo: 'original 2',
    print: print
})

if(typeof $$ !== 'undefined') {

    myClass.print();
    var overridden = overrideClass({
        propertyOne: 'overridden 1',
        propertyTwo: 'overridden 2'
    })
    overridden.print();
    var interfaced = myInterface(overridden);
    interfaced.print();
    overrideClass(interfaced).print();
    myInterface(myClass).print();
    
}

/* expected output
original 1
original 2
overridden 1
overridden 2
overridden 1
undefined
overridden 1
original 2
original 1
undefined
*/



original 1
original 2
overridden 1
overridden 2
overridden 1
undefined
overridden 1
original 2
original 1
undefined
